In [29]:
%load_ext autoreload
%autoreload 2

from FP567_Lib import *

import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib
import os
import glob
import json
import numpy as np
import pandas as pd

# improve/change plot appearance
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
'''
We have all the files in resources/market_item_data/, which are item infos in the form of:
{ 
    item_id : 
    [
        [date/time_n+1, price_at_time_n+1, amount_sold_at_time_n+1], [date/time_n+2, price_at_time_n+2, amount_sold_at_time_n+2], ..., [date/time_n+m, price_at_time_n+m, amount_sold_at_time_n+m]
    ]
}
for some amount of time.
The amount of time varies between items, as not all items have existed as long as others.

Let's make a Market object, which computes a bunch of the info we want.
'''
market = Market()

In [30]:
'''
So, lets extend all the items that are not highest_unit_time worth of info, backwards in time,
so that the items that do not have as many as highest_unit_time, now have highest_unit_time
amount of info, with the time stamp, but just 0, 0 for those days.

To do that, we can call the balance method of the Market object, giving it the 
longest time span of unix times, and 0, 0 as the default amount sold and price
'''
market.balance_as_is(0, 0)
market.is_balanced()

True